In [1]:
import os
import pandas as pd
import numpy as np
import glob
import warnings

In [2]:
path = "/mnt/data/shared/jacob/GFNO/results"
os.chdir(path)

In [3]:
def to_numeric(x):
    try:
        x = float(x)
        if x.is_integer():
            x = int(x)
        return x
    except ValueError:
        return x

def latex_sci(x, prec=3):
    return ("\\times10^{".join(f'{{:.{prec}e}}'.format(x).split("e")) + "}").replace("10^{-0", "10^{-")

def round_(num, prec):
    return f"{num:.{prec}f}"


# extract results for each model from a list of experiment files
'''
experiments: list of paths to results.txt
models: model names
n_rep: number of replicates of each experiment
'''
def extract_results(experiments, models, n_rep=3, summary=True, ten3=False):

    # add path sep to the end of the model names so that fno2d can be identified from fno2d_aug
    models = [os.path.join(mod, "") for mod in models]

    # init a results dictionary for the experiment
    results_exp = {k:None for k in models}

    # loop over each of the models; aggregate results for each model
    for model in models:

        # see which experiments contain the model in the title and make sure we have three replicates
        model_exps = [exp for exp in experiments if model in exp]
        if len(model_exps) == 0:
            results_exp[model] = None
            warnings.warn(f"No results for {model}; setting None")
            continue
        assert len(model_exps) > 0, f"{len(model_exps)} exps for model {model}"
        assert len(model_exps) == n_rep, f"{len(model_exps)} exps for model {model}; Experiments found: {model_exps}; Experiments: {experiments}"
        exps_str = " ".join(model_exps)
        assert "seed1" in exps_str, f"Missing seed1 in {exps_str}"
        assert "seed2" in exps_str, f"Missing seed2 in {exps_str}"
        assert "seed3" in exps_str, f"Missing seed3 in {exps_str}"

        # iterate over the experiments for the model to agg results
        results_mod = None
        epochs = None
        for model_exp in model_exps:
            with open(model_exp, "r") as f:
                result_mod_exp = f.read().splitlines()

            # convert the results to a dictionary; convert numerics to float or int
            result_mod_exp = {k: to_numeric(v) for k, v in [line.split(": ") for line in result_mod_exp]}

            # extract number of epochs
            if not epochs:
                args = result_mod_exp['Args'].split(", ")
                epochs = int([arg.split("=")[1] for arg in args if "epochs" in arg][0])

            # init the results dictionary for the model
            if results_mod is None:
                results_mod = {k:[] for k in result_mod_exp.keys()}

            # add the results for the experiment to the aggregated model results
            for k in result_mod_exp.keys():
                results_mod[k].append(result_mod_exp[k])

        if not summary: # return raw results
            return results_mod

        # get summary stats for each of the results
        for k in results_mod.keys():
            try:
                res_k = np.array(results_mod[k])

                # if the result is time, convert from seconds to train to seconds per epoch
                if k == "Train time":
                    res_k = res_k / epochs
                elif k in ['Train', 'Valid', 'Test', 'Rotation Test','Reflection Test',  'Super Space Test', 'Super Space Rotation Test', 'Super Time Test', 'Super Time Rotation Test', 'Best Valid', 'Super Space Interpolation Test', 'Super Time Interpolation Test', 'Mean inference time']:

                    # convert from decimal to percent
                    if ten3:
                        res_k = res_k * 1000
                    else:
                        res_k = res_k * 100

                # check for inf
                inf_bool = np.isinf(res_k)
                if inf_bool.any():
                    warnings.warn(f"Infinity detected and filtered: Key {k}, exp {model_exps[inf_bool.nonzero()[0][0]]}")
                    res_k = res_k[np.logical_not(inf_bool)]
                results_mod[k] = {"mean":res_k.mean(), "sd":res_k.std()}
            except:
                pass

        # add the results for the model to the experiment results
        results_exp[model] = results_mod

    # remove the path sep
    results_exp = {(k.split(os.sep)[0]):v for k, v in results_exp.items()}

    return results_exp

'''
experiments: list of paths to results.txt
models: model names
key: name of result to extract
prec: rounding precision
'''
def extract_key(experiments, models, key, prec=3, scientific=False, format=True, par_millions=False, ten3=False):

    # extract results for the experiment
    results_exp = extract_results(experiments, models, ten3=ten3)

    # remove None results
    results_exp2 = results_exp.copy()
    for k, v in results_exp.items():
        if v is None:
            del results_exp2[k]
            warnings.warn(f"Results for {k} are None; removing")
    results_exp = results_exp2

    # extract key for the experiment
    allNone = True
    for k in results_exp.keys(): # keys are model names
        if key in results_exp[k].keys():
            results_exp[k] = results_exp[k][key]
        else:
            results_exp[k] = {"mean":404, "sd":404}
        if allNone and results_exp[k] is not None:
            allNone = False
    if allNone:
        warnings.warn("No results found")
        return None

    # find which model has the best test error
    # return  results_exp
    sorted_mods = sorted({k: v["mean"] for k, v in results_exp.items()}.items(), key=lambda x: x[1])
    best_mod = sorted_mods[0][0]
    second_mod = sorted_mods[1][0]

    form = round_
    if scientific:
        form = latex_sci

    # format the test errors
    res = {k:None for k in results_exp.keys()}
    for k, v in results_exp.items(): # k are model names
        if key == "Parameters":
            if par_millions:
                res[k] = f"${round(v['mean'] / 1e6, prec):.{prec}f}$"
            else:
                res[k] = f"${int(v['mean']):,}$"
        elif key == "Train time":
            res[k] = round(v['mean'], prec)
        elif format:
            if k == best_mod:
                res[k] = f"$\\mathbf{{{form(v['mean'], prec)}}}({form(v['sd'], prec)})$"
            elif k == second_mod:
                res[k] = f"$\\underline{{{form(v['mean'], prec)}}}({form(v['sd'], prec)})$"
            else:
                res[k] = f"${form(v['mean'], prec)}({form(v['sd'], prec)})$"
        else:
            res[k] = v
    return res

def results_frame(exps, mods, keys, prec=3, scientific=False, format=True, summary_only=False, remove_dimension=False, par_millions=False, ten3=False):
    results = {key:extract_key(exps, mods, key, prec, scientific, format or summary_only, par_millions, ten3) for key in keys}
    if any([result is None for result in list(results.values())]):
        warnings.warn("Detected none; not returning dictionary")
        return results
    key_update = {'Train time':'Seconds/Epoch', 'Test':'Test (\\%)', 'Rotation Test':'Test\\textsubscript{$90^\circ$} (\\%)', 'Super Space Test':'Space Super-res. (\\%)', 'Super Time Test':'Space-Time Super-res. (\\%)', 'Super Space Interpolation Test':'Space Int. (\\%)', 'Super Time Interpolation Test':'Space-Time Int. (\\%)', 'Valid':'Valid (\\%)', 'Mean inference time':'Inference Time (ms)'}
    if par_millions:
        key_update["Parameters"] = "\\# Par. (M)"
    results = {(key_update[key] if key in key_update.keys() else key):result for key, result in results.items()}
    df = pd.DataFrame(results)
    df.index.names = ["Model"]
    df.index = [model.replace("_", "-").replace("GFNO", "$G$-FNO") for model in df.index]
    new_idx = []
    df.rename(index={"FNO2d-aug":"FNO2d+p4", "FNO2d-aug-rf":"FNO2d+p4m", "Unet-Rot-M20":"U-Net2d-p4", "Unet-Rot-M2d":"U-Net2d-p4",
                     "FNO3d-aug":"FNO3d+p4", "FNO3d-aug-rf":"FNO3d+p4m", "Unet-Rot-3D20":"U-Net3d-p4", "Unet-Rot32-3D":"U-Net3d-p4", "Unet-Rot-3D":"U-Net3d-p4",
                     "radialNO2d-p4":"radialFNO-p4", "radialNO2d-p4m":"radialFNO-p4m", "radialNO3d-p4":"radialFNO-p4", "radialNO3d-p4m":"radialFNO-p4m"}, inplace=True)
    for j in range(len(df.index)):
        new_name = df.index[j]
        if remove_dimension:
            new_name = new_name.replace('2d', '').replace('3d', '')
        if new_name[-2:] == "p4":
            new_idx.append(new_name.replace('p4', '$p4$'))
        elif new_name[-3:] == "p4m":
            new_idx.append(new_name.replace('p4m', '$p4m$'))
        else:
            new_idx.append(new_name)
    df.index = new_idx
    if not format:
        return df
    # format = "cp{3cm}" * (len(keys) + 1)
    format ="l" + "c" * (len(keys))
    return df.to_latex(escape=False, column_format=format)

In [4]:
datas = {"ns_sym":"ns_V0.0001_N1200_T30_cos4.mat", "swe":"2D_rdb_NA_NA.h5", "ns":"ns_V1e-4_N10000_T30.mat", "arena":"ShallowWater2D"}
exps = {data_name:glob.glob(data + '/**/*.txt', recursive=True) for data_name, data in datas.items()}
total_exps = len(sum(list(exps.values()), []))
print(f"Num exps: {total_exps}")

Num exps: 315


In [5]:
special_exps = ["markov", "steer", "recurrent", "hybrid", "nogrid", "symmetric", "cartesian"]
for exp in special_exps:
    exps[f"ns_{exp}"] = [ns_exp for ns_exp in exps["ns"] if exp in ns_exp]
    exps["ns"] = [ns_exp for ns_exp in exps["ns"] if exp not in ns_exp]

In [6]:
exp_ct = {data_name:len(exps) for data_name, exps in exps.items()}
assert sum(exp_ct.values()) == total_exps
exp_ct

{'ns_sym': 48,
 'swe': 48,
 'ns': 48,
 'arena': 51,
 'ns_markov': 24,
 'ns_steer': 6,
 'ns_recurrent': 24,
 'ns_hybrid': 18,
 'ns_nogrid': 24,
 'ns_symmetric': 9,
 'ns_cartesian': 15}

In [7]:
mods = sorted("FNO2d  FNO2d_aug  FNO2d_aug-rf  FNO3d  FNO3d_aug  FNO3d_aug-rf  GFNO2d_p4  GFNO2d_p4m  GFNO3d_p4  GFNO3d_p4m Unet_Rot_3D  Unet_Rot_M2d radialNO2d_p4 radialNO2d_p4m radialNO3d_p4 radialNO3d_p4m".split())
mods2d = [mod for mod in mods if "2d" in mod]
keys2d_full = ['Parameters', 'Test', 'Rotation Test', 'Super Space Test', 'Super Space Interpolation Test']
keys2d = ['Parameters', 'Test']
keys2d_rotation = ["Test", "Rotation Test"]
keys2d_super = ['Super Space Test', 'Super Space Interpolation Test']
mods3d = [mod for mod in mods if "3d" in mod or "3D" in mod]
keys3d_full = ['Parameters', 'Test', 'Rotation Test', 'Super Time Test', 'Super Time Interpolation Test']
keys3d = ['Parameters', 'Test']
keys3d_rotation = ["Test", "Rotation Test"]
keys3d_super = ['Super Time Test', 'Super Time Interpolation Test']
extract_results(exps['ns'], mods, 3)["Unet_Rot_M2d"].keys()

dict_keys(['Args', 'Parameters', 'Train time', 'Train', 'Valid', 'Test', 'Rotation Test', 'Reflection Test', 'Super Space Test', 'Super Space Interpolation Test', 'Super Space Rotation Test', 'Super Space Reflection Test', 'Super S', 'Super Time Test', 'Super Time Interpolation Test', 'Super Time Rotation Test', 'Super Time Reflection Test', 'Super T', 'Best Valid', 'Best epoch', 'Test Rotation Equivariance loss', 'Test Reflection Equivariance loss', 'Epochs trained'])

In [8]:
# archived tables
print(results_frame(exps["ns"], mods2d, keys2d_full, prec=2))

\begin{tabular}{lccccc}
\toprule
{} &   Parameters &                 Test (\%) & Test\textsubscript{$90^\circ$} (\%) &      Space Super-res. (\%) &            Space Int. (\%) \\
\midrule
FNO2d           &    $928,661$ &              $8.41(0.41)$ &                      $129.21(3.90)$ &     $\mathbf{43.02}(0.18)$ &     $\mathbf{43.14}(0.19)$ \\
FNO2d+$p4$      &    $928,661$ &             $10.44(0.47)$ &                       $10.38(0.38)$ &              $49.78(8.40)$ &              $43.72(0.45)$ \\
FNO2d+$p4m$     &    $928,661$ &             $22.09(1.46)$ &                       $22.61(1.54)$ &              $54.04(4.52)$ &              $46.30(1.33)$ \\
$G$-FNO2d-$p4$  &    $853,121$ &     $\mathbf{4.78}(0.39)$ &               $\mathbf{4.78}(0.39)$ &  $\underline{43.41}(0.12)$ &  $\underline{43.51}(0.11)$ \\
$G$-FNO2d-$p4m$ &    $835,969$ &  $\underline{6.19}(0.61)$ &            $\underline{6.19}(0.61)$ &              $43.78(0.33)$ &              $43.88(0.30)$ \\
U-Net2d-$p4$    &  $3,6

/tmp/ipykernel_451806/3431984908.py:200: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return df.to_latex(escape=False, column_format=format)


In [9]:
# NS Main Results

ns2d = results_frame(exps["ns"], mods2d, keys2d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns3d = results_frame(exps["ns"], mods3d, keys2d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym2d = results_frame(exps["ns_sym"], mods2d, keys3d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym3d = results_frame(exps["ns_sym"], mods3d, keys3d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)

# ns2d.rename({"Test (\%)":"NS"}, axis=1, inplace=True)
# ns_sym2d.rename({"Test (\%)":"NS-sym"}, axis=1, inplace=True)
# ns3d.rename({"Test (\%)":"NS"}, axis=1, inplace=True)
# ns_sym3d.rename({"Test (\%)":"NS-sym"}, axis=1, inplace=True)
#
#
# ns2d_full = pd.concat([ns2d, ns_sym2d.drop("\# Par. (M)", axis=1)], axis=1)
# ns3d_full = pd.concat([ns3d, ns_sym3d.drop("\# Par. (M)", axis=1)], axis=1)

ns2d_full = pd.concat({"NS":ns2d, "NS-sym":ns_sym2d}, axis=1)
ns3d_full = pd.concat({"NS":ns3d, "NS-sym":ns_sym3d}, axis=1)

ns_full = pd.concat({"2D Models":ns2d_full, "3D Models":ns3d_full}, axis=1)

In [10]:
ns_full

2D Models                                        \
                         NS                                NS-sym   
                \# Par. (M)                 Test (\%) \# Par. (M)   
FNO                  $0.93$              $8.41(0.41)$      $0.93$   
FNO+$p4$             $0.93$             $10.44(0.47)$      $0.93$   
FNO+$p4m$            $0.93$             $22.09(1.46)$      $0.93$   
$G$-FNO-$p4$         $0.85$     $\mathbf{4.78}(0.39)$      $0.85$   
$G$-FNO-$p4m$        $0.84$  $\underline{6.19}(0.61)$      $0.84$   
U-Net-$p4$           $3.65$             $18.40(0.44)$      $3.65$   
radialFNO-$p4$       $1.03$              $9.21(0.26)$      $1.03$   
radialFNO-$p4m$      $0.95$             $10.86(0.18)$      $0.95$   

                                            3D Models  \
                                                   NS   
                                Test (\%) \# Par. (M)   
FNO                          $4.21(0.12)$      $4.92$   
FNO+$p4$                     $4.80(0.12)$      $4.92$   
FNO+$p4m$                   $13.06(3.29)$      $4.92$   
$G$-FNO-$p4$        $\mathbf{2.24}(0.09)$      $4.80$   
$G$-FNO-$p4m$    $\underline{2.37}(0.19)$      $3.89$   
U-Net-$p4$                  $15.39(0.16)$      $6.08$   
radialFNO-$p4$              $12.81(0.42)$      $4.98$   
radialFNO-$p4m$             $17.39(0.22)$      $5.63$   

                                                        \
                                                NS-sym   
                                 Test (\%) \# Par. (M)   
FNO                          $15.84(0.37)$      $4.92$   
FNO+$p4$                     $14.14(0.14)$      $4.92$   
FNO+$p4m$                    $15.32(0.05)$      $4.92$   
$G$-FNO-$p4$        $\mathbf{11.77}(0.13)$      $4.80$   
$G$-FNO-$p4m$                $12.71(0.31)$      $3.89$   
U-Net-$p4$                   $24.62(0.29)$      $6.08$   
radialFNO-$p4$               $12.09(0.08)$      $4.98$   
radialFNO-$p4m$  $\underline{11.83}(0.23)$      $5.63$   

                                            
                                            
                                 Test (\%)  
FNO                          $26.02(0.41)$  
FNO+$p4$         $\underline{15.75}(0.33)$  
FNO+$p4m$                    $22.25(0.21)$  
$G$-FNO-$p4$        $\mathbf{10.72}(0.27)$  
$G$-FNO-$p4m$                $17.21(1.35)$  
U-Net-$p4$                   $21.82(0.10)$  
radialFNO-$p4$               $17.54(0.60)$  
radialFNO-$p4m$              $17.27(0.17)$

In [11]:
# par_len = 1.7
# main_len = 1.9
# format = "l" + "|".join([f">{{\\centering\\arraybackslash}}m{{{par_len}cm}}>{{\\centering\\arraybackslash}}m{{{main_len}cm}}>{{\\centering\\arraybackslash}}m{{{main_len}cm}}"] * 2)
format = "l" + "|".join(["c" * 4] * 2)
print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcccc|cccc}
\toprule
{} & \multicolumn{4}{c}{2D Models} & \multicolumn{4}{c}{3D Models} \\
{} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} \\
{} & \# Par. (M) &                 Test (\%) & \# Par. (M) &                 Test (\%) & \# Par. (M) &                  Test (\%) & \# Par. (M) &                  Test (\%) \\
\midrule
FNO             &      $0.93$ &              $8.41(0.41)$ &      $0.93$ &              $4.21(0.12)$ &      $4.92$ &              $15.84(0.37)$ &      $4.92$ &              $26.02(0.41)$ \\
FNO+$p4$        &      $0.93$ &             $10.44(0.47)$ &      $0.93$ &              $4.80(0.12)$ &      $4.92$ &              $14.14(0.14)$ &      $4.92$ &  $\underline{15.75}(0.33)$ \\
FNO+$p4m$       &      $0.93$ &             $22.09(1.46)$ &      $0.93$ &             $13.06(3.29)$ &      $4.92$ &              $15.32(0.05)$ &      $4.92$ &              $22.25(0.21)$ \\
$G$-FNO-$p4$    &      $0.8

/tmp/ipykernel_3755212/469565081.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


In [12]:
# NS Super resolution

ns2d = results_frame(exps["ns"], mods2d, keys2d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns3d = results_frame(exps["ns"], mods3d, keys3d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym2d = results_frame(exps["ns_sym"], mods2d, keys2d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym3d = results_frame(exps["ns_sym"], mods3d, keys3d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)

ns2d.rename({"Space Super-res. (\\%)":"SR Test (\\%)", "Space Int. (\\%)":"Int. Test (\\%)"}, axis=1, inplace=True)
ns_sym2d.rename({"Space Super-res. (\\%)":"SR Test (\\%)", "Space Int. (\\%)":"Int. Test (\\%)"}, axis=1, inplace=True)
ns3d.rename({"Space-Time Super-res. (\\%)":"SR Test (\\%)", "Space-Time Int. (\\%)":"Int. Test (\\%)"}, axis=1, inplace=True)
ns_sym3d.rename({"Space-Time Super-res. (\\%)":"SR Test (\\%)", "Space-Time Int. (\\%)":"Int. Test (\\%)"}, axis=1, inplace=True)


ns2d_full = pd.concat({"NS":ns2d, "NS-sym":ns_sym2d}, axis=1)
ns3d_full = pd.concat({"NS":ns3d, "NS-sym":ns_sym3d}, axis=1)

ns_full = pd.concat({"2D Models":ns2d_full, "3D Models":ns3d_full}, axis=1)

In [13]:
ns_full

2D Models                             \
                                        NS                              
                              SR Test (\%)             Int. Test (\%)   
FNO                 $\mathbf{43.02}(0.18)$     $\mathbf{43.14}(0.19)$   
FNO+$p4$                     $49.78(8.40)$              $43.72(0.45)$   
FNO+$p4m$                    $54.04(4.52)$              $46.30(1.33)$   
$G$-FNO-$p4$     $\underline{43.41}(0.12)$  $\underline{43.51}(0.11)$   
$G$-FNO-$p4m$                $43.78(0.33)$              $43.88(0.30)$   
U-Net-$p4$                   $92.00(7.22)$              $43.68(0.82)$   
radialFNO-$p4$               $43.73(0.07)$              $43.86(0.07)$   
radialFNO-$p4m$              $43.91(0.69)$              $44.02(0.71)$   

                                                                       \
                                    NS-sym                              
                              SR Test (\%)             Int. Test (\%)   
FNO                          $32.45(1.47)$  $\underline{23.33}(0.07)$   
FNO+$p4$                     $31.72(1.55)$     $\mathbf{23.32}(0.09)$   
FNO+$p4m$                    $32.68(0.84)$              $25.02(1.24)$   
$G$-FNO-$p4$        $\mathbf{21.89}(0.05)$              $23.36(0.04)$   
$G$-FNO-$p4m$    $\underline{22.09}(0.03)$              $23.56(0.03)$   
U-Net-$p4$                   $70.42(1.66)$              $24.24(0.12)$   
radialFNO-$p4$               $25.47(0.31)$              $26.52(0.35)$   
radialFNO-$p4m$              $27.85(0.22)$              $28.70(0.05)$   

                                 3D Models                             \
                                        NS                              
                              SR Test (\%)             Int. Test (\%)   
FNO                          $29.99(0.26)$              $27.97(0.10)$   
FNO+$p4$                     $30.36(0.18)$  $\underline{27.27}(0.11)$   
FNO+$p4m$                    $30.45(0.37)$              $27.82(0.12)$   
$G$-FNO-$p4$        $\mathbf{29.62}(0.15)$     $\mathbf{27.09}(0.06)$   
$G$-FNO-$p4m$                $30.02(0.31)$              $27.38(0.21)$   
U-Net-$p4$                 $114.99(33.94)$              $30.11(0.41)$   
radialFNO-$p4$   $\underline{29.92}(0.65)$              $27.52(0.48)$   
radialFNO-$p4m$              $29.94(0.60)$              $27.40(0.45)$   

                                                                       
                                    NS-sym                             
                              SR Test (\%)             Int. Test (\%)  
FNO                          $31.24(0.66)$              $29.38(0.47)$  
FNO+$p4$                     $25.25(0.80)$  $\underline{21.20}(0.31)$  
FNO+$p4m$                    $31.44(1.20)$              $26.64(0.32)$  
$G$-FNO-$p4$        $\mathbf{20.44}(0.73)$     $\mathbf{17.71}(0.09)$  
$G$-FNO-$p4m$    $\underline{23.98}(1.30)$              $22.14(0.96)$  
U-Net-$p4$                   $79.08(3.60)$              $25.85(0.03)$  
radialFNO-$p4$               $24.87(0.51)$              $22.83(0.42)$  
radialFNO-$p4m$              $24.49(0.14)$              $22.60(0.14)$

In [14]:
format = "l" + "|".join(["c" * 4] * 2)
print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcccc|cccc}
\toprule
{} & \multicolumn{4}{c}{2D Models} & \multicolumn{4}{c}{3D Models} \\
{} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} \\
{} &               SR Test (\%) &             Int. Test (\%) &               SR Test (\%) &             Int. Test (\%) &               SR Test (\%) &             Int. Test (\%) &               SR Test (\%) &             Int. Test (\%) \\
\midrule
FNO             &     $\mathbf{43.02}(0.18)$ &     $\mathbf{43.14}(0.19)$ &              $32.45(1.47)$ &  $\underline{23.33}(0.07)$ &              $29.99(0.26)$ &              $27.97(0.10)$ &              $31.24(0.66)$ &              $29.38(0.47)$ \\
FNO+$p4$        &              $49.78(8.40)$ &              $43.72(0.45)$ &              $31.72(1.55)$ &     $\mathbf{23.32}(0.09)$ &              $30.36(0.18)$ &  $\underline{27.27}(0.11)$ &              $25.25(0.80)$ &  $\underline{21.20}(0.31)$ \\
FNO+$p4m$       &             

/tmp/ipykernel_3755212/3650052898.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


In [15]:
# NS rotation

ns2d = results_frame(exps["ns"], mods2d, keys2d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns3d = results_frame(exps["ns"], mods3d, keys3d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym2d = results_frame(exps["ns_sym"], mods2d, keys2d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
ns_sym3d = results_frame(exps["ns_sym"], mods3d, keys3d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)

ns2d_full = pd.concat({"NS":ns2d, "NS-sym":ns_sym2d}, axis=1)
ns3d_full = pd.concat({"NS":ns3d, "NS-sym":ns_sym3d}, axis=1)

ns_full = pd.concat({"2D Models":ns2d_full, "3D Models":ns3d_full}, axis=1)

In [16]:
ns_full

2D Models                                      \
                                       NS                                       
                                Test (\%) Test\textsubscript{$90^\circ$} (\%)   
FNO                          $8.41(0.41)$                      $129.21(3.90)$   
FNO+$p4$                    $10.44(0.47)$                       $10.38(0.38)$   
FNO+$p4m$                   $22.09(1.46)$                       $22.61(1.54)$   
$G$-FNO-$p4$        $\mathbf{4.78}(0.39)$               $\mathbf{4.78}(0.39)$   
$G$-FNO-$p4m$    $\underline{6.19}(0.61)$            $\underline{6.19}(0.61)$   
U-Net-$p4$                  $18.40(0.44)$                       $18.40(0.44)$   
radialFNO-$p4$               $9.21(0.26)$                        $9.21(0.26)$   
radialFNO-$p4m$             $10.86(0.18)$                       $10.86(0.18)$   

                                                                               \
                                   NS-sym                                       
                                Test (\%) Test\textsubscript{$90^\circ$} (\%)   
FNO                          $4.21(0.12)$                        $9.91(0.90)$   
FNO+$p4$                     $4.80(0.12)$                        $4.74(0.20)$   
FNO+$p4m$                   $13.06(3.29)$                       $12.81(2.80)$   
$G$-FNO-$p4$        $\mathbf{2.24}(0.09)$               $\mathbf{2.24}(0.09)$   
$G$-FNO-$p4m$    $\underline{2.37}(0.19)$            $\underline{2.37}(0.19)$   
U-Net-$p4$                  $15.39(0.16)$                       $15.39(0.16)$   
radialFNO-$p4$              $12.81(0.42)$                       $12.81(0.42)$   
radialFNO-$p4m$             $17.39(0.22)$                       $17.39(0.22)$   

                                 3D Models  \
                                        NS   
                                 Test (\%)   
FNO                          $15.84(0.37)$   
FNO+$p4$                     $14.14(0.14)$   
FNO+$p4m$                    $15.32(0.05)$   
$G$-FNO-$p4$        $\mathbf{11.77}(0.13)$   
$G$-FNO-$p4m$                $12.71(0.31)$   
U-Net-$p4$                   $24.62(0.29)$   
radialFNO-$p4$               $12.09(0.08)$   
radialFNO-$p4m$  $\underline{11.83}(0.23)$   

                                                     \
                                                      
                Test\textsubscript{$90^\circ$} (\%)   
FNO                                  $100.75(2.20)$   
FNO+$p4$                              $14.21(0.15)$   
FNO+$p4m$                             $15.37(0.03)$   
$G$-FNO-$p4$                 $\mathbf{11.77}(0.13)$   
$G$-FNO-$p4m$                         $12.71(0.31)$   
U-Net-$p4$                            $24.62(0.29)$   
radialFNO-$p4$                        $12.09(0.08)$   
radialFNO-$p4m$           $\underline{11.83}(0.23)$   

                                                                                
                                    NS-sym                                      
                                 Test (\%) Test\textsubscript{$90^\circ$} (\%)  
FNO                          $26.02(0.41)$                       $26.75(0.64)$  
FNO+$p4$         $\underline{15.75}(0.33)$           $\underline{15.85}(0.31)$  
FNO+$p4m$                    $22.25(0.21)$                       $22.24(0.20)$  
$G$-FNO-$p4$        $\mathbf{10.72}(0.27)$              $\mathbf{10.72}(0.27)$  
$G$-FNO-$p4m$                $17.21(1.35)$                       $17.21(1.35)$  
U-Net-$p4$                   $21.82(0.10)$                       $21.82(0.10)$  
radialFNO-$p4$               $17.54(0.60)$                       $17.54(0.60)$  
radialFNO-$p4m$              $17.27(0.17)$                       $17.27(0.17)$

In [17]:
format = "l" + "|".join(["c" * 4] * 2)
print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcccc|cccc}
\toprule
{} & \multicolumn{4}{c}{2D Models} & \multicolumn{4}{c}{3D Models} \\
{} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} & \multicolumn{2}{c}{NS} & \multicolumn{2}{c}{NS-sym} \\
{} &                 Test (\%) & Test\textsubscript{$90^\circ$} (\%) &                 Test (\%) & Test\textsubscript{$90^\circ$} (\%) &                  Test (\%) & Test\textsubscript{$90^\circ$} (\%) &                  Test (\%) & Test\textsubscript{$90^\circ$} (\%) \\
\midrule
FNO             &              $8.41(0.41)$ &                      $129.21(3.90)$ &              $4.21(0.12)$ &                        $9.91(0.90)$ &              $15.84(0.37)$ &                      $100.75(2.20)$ &              $26.02(0.41)$ &                       $26.75(0.64)$ \\
FNO+$p4$        &             $10.44(0.47)$ &                       $10.38(0.38)$ &              $4.80(0.12)$ &                        $4.74(0.20)$ &              $14.14(0.14)$ &                       $14.21(0.15)

/tmp/ipykernel_3755212/3650052898.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ns_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


In [18]:
swe3d = [mod for mod in mods3d if mod != 'Unet_Rot_3D'] + ['Unet_Rot32_3D']

In [19]:
# SWE Main results

arena2d = results_frame(exps["arena"], mods2d, keys2d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
arena3d = results_frame(exps["arena"], swe3d, keys2d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
swe2d = results_frame(exps["swe"], mods2d, keys3d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)
swe3d = results_frame(exps["swe"], mods3d, keys3d, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)

# arena3d = swe3d.copy()
# arena3d.iloc[:, 1] = np.nan

swe2d.rename({"Test (\\%)":"Test ($\\times10^{-3}$)"}, axis=1, inplace=True)
swe3d.rename({"Test (\\%)":"Test ($\\times10^{-3}$)"}, axis=1, inplace=True)


swe2d_full = pd.concat({"SWE-arena":arena2d, "SWE-bench":swe2d}, axis=1)
swe3d_full = pd.concat({"SWE-arena":arena3d, "SWE-bench":swe3d}, axis=1)

swe_full = pd.concat({"2D Models": swe2d_full, "3D Models":swe3d_full},axis=1)

In [20]:
swe_full

2D Models                                         \
                  SWE-arena                              SWE-bench   
                \# Par. (M)                  Test (\%) \# Par. (M)   
FNO                  $6.56$              $18.45(0.89)$      $0.93$   
FNO+$p4$             $6.56$              $33.08(0.20)$      $0.93$   
FNO+$p4m$            $6.56$              $44.24(1.63)$      $0.93$   
$G$-FNO-$p4$         $6.36$     $\mathbf{14.96}(0.06)$      $0.85$   
$G$-FNO-$p4m$        $6.23$  $\underline{16.20}(0.17)$      $0.84$   
U-Net-$p4$           $6.90$              $28.79(0.08)$      $3.65$   
radialFNO-$p4$       $6.79$              $23.37(0.14)$      $1.03$   
radialFNO-$p4m$      $6.84$              $26.20(0.55)$      $0.95$   

                                            3D Models  \
                                            SWE-arena   
                   Test ($\times10^{-3}$) \# Par. (M)   
FNO                          $1.22(0.07)$     $49.57$   
FNO+$p4$                     $1.33(0.08)$     $49.57$   
FNO+$p4m$                    $1.32(0.07)$     $49.57$   
$G$-FNO-$p4$                 $1.21(0.02)$     $53.70$   
$G$-FNO-$p4m$                $1.11(0.17)$     $56.81$   
U-Net-$p4$                 $30.86(11.31)$      $6.08$   
radialFNO-$p4$   $\underline{0.71}(0.03)$     $53.40$   
radialFNO-$p4m$     $\mathbf{0.70}(0.07)$     $51.92$   

                                                        \
                                             SWE-bench   
                                 Test (\%) \# Par. (M)   
FNO                 $\mathbf{41.49}(0.12)$      $4.92$   
FNO+$p4$                     $44.16(0.14)$      $4.92$   
FNO+$p4m$                    $45.06(0.20)$      $4.92$   
$G$-FNO-$p4$                 $43.68(0.55)$      $4.80$   
$G$-FNO-$p4m$    $\underline{43.46}(0.79)$      $3.89$   
U-Net-$p4$                   $55.28(1.86)$      $6.08$   
radialFNO-$p4$               $44.12(0.20)$      $4.98$   
radialFNO-$p4m$              $44.76(0.15)$      $5.63$   

                                           
                                           
                   Test ($\times10^{-3}$)  
FNO                          $1.59(0.02)$  
FNO+$p4$                     $1.64(0.02)$  
FNO+$p4m$                    $1.65(0.03)$  
$G$-FNO-$p4$     $\underline{1.44}(0.02)$  
$G$-FNO-$p4m$       $\mathbf{1.44}(0.02)$  
U-Net-$p4$                   $8.03(0.35)$  
radialFNO-$p4$               $1.54(0.02)$  
radialFNO-$p4m$              $1.49(0.01)$

In [21]:
format = "l" + "|".join(["c" * 4] * 2)
print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcccc|cccc}
\toprule
{} & \multicolumn{4}{c}{2D Models} & \multicolumn{4}{c}{3D Models} \\
{} & \multicolumn{2}{c}{SWE-arena} & \multicolumn{2}{c}{SWE-bench} & \multicolumn{2}{c}{SWE-arena} & \multicolumn{2}{c}{SWE-bench} \\
{} & \# Par. (M) &                  Test (\%) & \# Par. (M) &    Test ($\times10^{-3}$) & \# Par. (M) &                  Test (\%) & \# Par. (M) &    Test ($\times10^{-3}$) \\
\midrule
FNO             &      $6.56$ &              $18.45(0.89)$ &      $0.93$ &              $1.22(0.07)$ &     $49.57$ &     $\mathbf{41.49}(0.12)$ &      $4.92$ &              $1.59(0.02)$ \\
FNO+$p4$        &      $6.56$ &              $33.08(0.20)$ &      $0.93$ &              $1.33(0.08)$ &     $49.57$ &              $44.16(0.14)$ &      $4.92$ &              $1.64(0.02)$ \\
FNO+$p4m$       &      $6.56$ &              $44.24(1.63)$ &      $0.93$ &              $1.32(0.07)$ &     $49.57$ &              $45.06(0.20)$ &      $4.92$ &              $1.65(0.03)$ \\
$G$-FNO

/tmp/ipykernel_3755212/3089597209.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


In [10]:
# SWE Super resolution

swe2d = results_frame(exps["swe"], mods2d, keys2d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)
swe3d = results_frame(exps["swe"], mods3d, keys3d_super, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)

swe2d.rename({"Space Super-res. (\\%)":"SR Test ($\\times10^{-3}$)", "Space Int. (\\%)":"Int. Test ($\\times10^{-3}$)"}, axis=1, inplace=True)
swe3d.rename({"Space-Time Super-res. (\\%)":"SR Test ($\\times10^{-3}$)", "Space-Time Int. (\\%)":"Int. Test ($\\times10^{-3}$)"}, axis=1, inplace=True)

swe_full = pd.concat({"2D Models":swe2d, "3D Models":swe3d}, axis=1)

In [11]:
swe_full

2D Models                              \
                 SR Test ($\times10^{-3}$) Int. Test ($\times10^{-3}$)   
FNO              $\underline{15.56}(2.92)$               $16.15(0.01)$   
FNO+$p4$            $\mathbf{14.80}(3.30)$               $16.16(0.01)$   
FNO+$p4m$                    $16.38(3.94)$               $16.16(0.01)$   
$G$-FNO-$p4$                 $19.39(4.60)$               $16.15(0.00)$   
$G$-FNO-$p4m$               $31.00(11.27)$               $16.15(0.01)$   
U-Net-$p4$              $3009.65(2348.93)$               $35.19(9.48)$   
radialFNO-$p4$               $30.40(4.20)$   $\underline{16.12}(0.00)$   
radialFNO-$p4m$              $22.42(0.95)$      $\mathbf{16.12}(0.00)$   

                                 3D Models                              
                 SR Test ($\times10^{-3}$) Int. Test ($\times10^{-3}$)  
FNO                          $16.90(0.58)$               $17.76(0.01)$  
FNO+$p4$                     $17.19(0.86)$               $17.76(0.01)$  
FNO+$p4m$                    $17.11(0.83)$               $17.75(0.01)$  
$G$-FNO-$p4$                 $15.87(0.60)$      $\mathbf{17.75}(0.00)$  
$G$-FNO-$p4m$                $16.88(1.19)$   $\underline{17.75}(0.00)$  
U-Net-$p4$                 $167.76(36.77)$               $19.19(0.13)$  
radialFNO-$p4$   $\underline{14.48}(0.76)$               $17.76(0.00)$  
radialFNO-$p4m$     $\mathbf{13.36}(0.36)$               $17.76(0.00)$

In [12]:
format = "l" + "|".join(["c" * 2] * 2)
print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcc|cc}
\toprule
{} & \multicolumn{2}{c}{2D Models} & \multicolumn{2}{c}{3D Models} \\
{} &  SR Test ($\times10^{-3}$) & Int. Test ($\times10^{-3}$) &  SR Test ($\times10^{-3}$) & Int. Test ($\times10^{-3}$) \\
\midrule
FNO             &  $\underline{15.56}(2.92)$ &               $16.15(0.01)$ &              $16.90(0.58)$ &               $17.76(0.01)$ \\
FNO+$p4$        &     $\mathbf{14.80}(3.30)$ &               $16.16(0.01)$ &              $17.19(0.86)$ &               $17.76(0.01)$ \\
FNO+$p4m$       &              $16.38(3.94)$ &               $16.16(0.01)$ &              $17.11(0.83)$ &               $17.75(0.01)$ \\
$G$-FNO-$p4$    &              $19.39(4.60)$ &               $16.15(0.00)$ &              $15.87(0.60)$ &      $\mathbf{17.75}(0.00)$ \\
$G$-FNO-$p4m$   &             $31.00(11.27)$ &               $16.15(0.01)$ &              $16.88(1.19)$ &   $\underline{17.75}(0.00)$ \\
U-Net-$p4$      &         $3009.65(2348.93)$ &               $35.19(9.48)$ &   

/tmp/ipykernel_451806/3007986941.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


In [14]:
# SWE Rotation

swe2d = results_frame(exps["swe"], mods2d, keys2d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)
swe3d = results_frame(exps["swe"], mods3d, keys3d_rotation, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True, ten3=True)

swe2d.rename({"Test\\textsubscript{$90^\circ$} (\%)":"Test\textsubscript{$90^\circ$} ($\\times10^{-3}$)", "Test (\%)":"Test ($\\times10^{-3}$)"}, axis=1, inplace=True)
swe3d.rename({"Test\\textsubscript{$90^\circ$} (\%)":"Test\textsubscript{$90^\circ$} ($\\times10^{-3}$)", "Test (\%)":"Test ($\\times10^{-3}$)"}, axis=1, inplace=True)

swe_full = pd.concat({"2D Models":swe2d, "3D Models":swe3d}, axis=1)

In [15]:
format = "l" + "|".join(["c" * 2] * 2)
print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))

\begin{tabular}{lcc|cc}
\toprule
{} & \multicolumn{2}{c}{2D Models} & \multicolumn{2}{c}{3D Models} \\
{} &    Test ($\times10^{-3}$) & Test\textsubscript{$90^\circ$} ($\times10^{-3}$) &    Test ($\times10^{-3}$) & Test\textsubscript{$90^\circ$} ($\times10^{-3}$) \\
\midrule
FNO             &              $1.22(0.07)$ &                                     $1.50(0.05)$ &              $1.59(0.02)$ &                                     $1.80(0.02)$ \\
FNO+$p4$        &              $1.33(0.08)$ &                                     $1.34(0.10)$ &              $1.64(0.02)$ &                                     $1.69(0.03)$ \\
FNO+$p4m$       &              $1.32(0.07)$ &                                     $1.33(0.07)$ &              $1.65(0.03)$ &                                     $1.71(0.03)$ \\
$G$-FNO-$p4$    &              $1.21(0.02)$ &                                     $1.21(0.02)$ &  $\underline{1.44}(0.02)$ &                         $\underline{1.44}(0.02)$ \\
$G$-FNO-$p4m$   

/tmp/ipykernel_451806/3007986941.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(swe_full.to_latex(escape=False, column_format=format, multicolumn_format="c"))


# G Hybrid

In [27]:
exps.keys()

dict_keys(['ns_sym', 'swe', 'ns', 'arena', 'ns_markov', 'ns_steer', 'ns_recurrent', 'ns_hybrid', 'ns_nogrid', 'ns_symmetric', 'ns_cartesian'])

In [28]:
hyb_exps = exps['ns_hybrid'] + [exp for exp in exps['ns'] if 'FNO2d/' in exp or 'GFNO2d' in exp]
hyb_exps_p4 = [exp for exp in hyb_exps if ('p4' in exp and 'p4m' not in exp) or 'FNO2d/' in exp]
hyb_exps_p4m = [exp for exp in hyb_exps if 'p4m' in exp or 'FNO2d/' in exp]

hybp4_mods = ["FNO2d", "Ghybrid2d_p41", "Ghybrid2d_p42", "Ghybrid2d_p43", "GFNO2d_p4"]
hybp4m_mods = ["FNO2d", "Ghybrid2d_p4m1", "Ghybrid2d_p4m2", "Ghybrid2d_p4m3", "GFNO2d_p4m"]

def hybrid_index(models):
    n_equiv = []
    for j in range(len(models)):
        if 'Ghybrid-p4m' in models[j]:
            n_equiv.append(models[j][-1])
            # models[j] = f"$G$-hybrid-$p4m$-{n_equiv[-1]}"
            models[j] = n_equiv[-1]
        elif 'Ghybrid-p4' in models[j]:
            n_equiv.append(models[j][-1])
            # models[j] = f"$G$-hybrid-$p4$-{n_equiv[-1]}"
            models[j] = n_equiv[-1]
        else:
            n_equiv.append(4 * ('p4' in models[j]))  # remaining models are FNO or GFNO
    return models, n_equiv

In [29]:
keys_hyb = ['Parameters'] + keys2d_rotation
hybp4_df = results_frame(hyb_exps_p4, hybp4_mods, keys_hyb, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
models = list(hybp4_df.index)
models, n_equiv = hybrid_index(models)
hybp4_df.index = n_equiv

hybp4m_df = results_frame(hyb_exps_p4m, hybp4m_mods, keys_hyb, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)
models = list(hybp4m_df.index)
models, n_equiv = hybrid_index(models)
hybp4m_df.index = n_equiv

n_eq = pd.DataFrame({"\# $G$-Fourier Layers": n_equiv})
n_eq.index = n_equiv

hyb_full = pd.concat({"":n_eq, "$p4$": hybp4_df, "$p4m$": hybp4m_df}, axis=1)


In [30]:
hyb_full

$p4$                            \
  \# $G$-Fourier Layers \# Par. (M)                 Test (\%)   
0                     0      $0.93$              $8.41(0.41)$   
1                     1      $1.14$              $7.17(0.51)$   
2                     2      $1.12$              $6.30(0.82)$   
3                     3      $1.10$  $\underline{6.12}(0.44)$   
4                     4      $0.85$     $\mathbf{4.78}(0.39)$   

                                            $p4m$                            \
  Test\textsubscript{$90^\circ$} (\%) \# Par. (M)                 Test (\%)   
0                      $129.21(3.90)$      $0.93$              $8.41(0.41)$   
1          $\underline{126.53}(3.66)$      $1.32$  $\underline{7.04}(0.41)$   
2                      $127.89(4.99)$      $1.30$              $7.32(0.37)$   
3                      $128.73(7.78)$      $1.27$              $7.29(0.14)$   
4               $\mathbf{4.78}(0.39)$      $0.84$     $\mathbf{6.19}(0.61)$   

                                       
  Test\textsubscript{$90^\circ$} (\%)  
0                      $129.21(3.90)$  
1                      $128.32(1.83)$  
2                      $129.15(3.02)$  
3          $\underline{125.74}(9.08)$  
4               $\mathbf{6.19}(0.61)$

In [31]:
format = "l" + "|".join(["c" * 3] * 2)
print(hyb_full.to_latex(escape=False, column_format=format, multicolumn_format="c", index=False))

\begin{tabular}{lccc|ccc}
\toprule
                      & \multicolumn{3}{c}{$p4$} & \multicolumn{3}{c}{$p4m$} \\
\# $G$-Fourier Layers & \# Par. (M) &                Test (\%) & Test\textsubscript{$90^\circ$} (\%) & \# Par. (M) &                Test (\%) & Test\textsubscript{$90^\circ$} (\%) \\
\midrule
                    0 &      $0.93$ &             $8.41(0.41)$ &                      $129.21(3.90)$ &      $0.93$ &             $8.41(0.41)$ &                      $129.21(3.90)$ \\
                    1 &      $1.14$ &             $7.17(0.51)$ &          $\underline{126.53}(3.66)$ &      $1.32$ & $\underline{7.04}(0.41)$ &                      $128.32(1.83)$ \\
                    2 &      $1.12$ &             $6.30(0.82)$ &                      $127.89(4.99)$ &      $1.30$ &             $7.32(0.37)$ &                      $129.15(3.02)$ \\
                    3 &      $1.10$ & $\underline{6.12}(0.44)$ &                      $128.73(7.78)$ &      $1.27$ &             $7.29(0.14)$ & 

/tmp/ipykernel_3755212/1881771958.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(hyb_full.to_latex(escape=False, column_format=format, multicolumn_format="c", index=False))


# Training Strategy

In [26]:
exps.keys()

dict_keys(['ns_sym', 'swe', 'ns', 'arena', 'ns_markov', 'ns_steer', 'ns_recurrent', 'ns_hybrid', 'ns_nogrid', 'ns_symmetric', 'ns_cartesian'])

In [27]:
tune_keys = ["Valid", "Super Space Test"]
# collect experiments by strategy
strategy_mods = mods2d
markov_exps = exps['ns_markov']
recurrent_exps = exps['ns_recurrent']
tf_exps = [exp for exp in exps['ns'] if '2d' in exp]

In [28]:
strategy_exps = {"Markov": markov_exps, "Recurrent": recurrent_exps, "Teacher Forcing": tf_exps}
strategy_dfs = {strategy: results_frame(exps, strategy_mods, tune_keys, format=0, par_millions=True, remove_dimension=True) for strategy, exps in strategy_exps.items()}

In [29]:
strategy_dfs['Markov']

,Valid (\%),Space Super-res. (\%)
FNO,"{'mean': 7.185795187950134, 'sd': 0.2760095831...","{'mean': 61.10313882430395, 'sd': 2.5842946801..."
FNO+$p4$,"{'mean': 10.899759729703268, 'sd': 0.239710071...","{'mean': 60.4571396112442, 'sd': 2.92262616185..."
FNO+$p4m$,"{'mean': 18.26482582092285, 'sd': 0.7529820869...","{'mean': 60.41047250231107, 'sd': 2.2827869178..."
$G$-FNO-$p4$,"{'mean': 5.0603160460789995, 'sd': 0.040300243...","{'mean': 43.34861770272255, 'sd': 0.2436138668..."
$G$-FNO-$p4m$,"{'mean': 6.58684515953064, 'sd': 0.95766801388...","{'mean': 43.180257777372994, 'sd': 0.822311925..."
U-Net-$p4$,"{'mean': 33.6953550974528, 'sd': 3.99875334289...","{'mean': 110.59697926044464, 'sd': 0.920359098..."
radialFNO-$p4$,"{'mean': 8.430923223495483, 'sd': 0.2479825729...","{'mean': 43.14296081662178, 'sd': 0.2872593572..."
radialFNO-$p4m$,"{'mean': 10.38261624177297, 'sd': 0.1874927634...","{'mean': 43.74323094884554, 'sd': 0.0887487829..."


In [30]:
for strategy in strategy_exps.keys():
    strategy_dfs[strategy]['Strategy'] = strategy
    strategy_dfs[strategy] = strategy_dfs[strategy][["Strategy", "Valid (\\%)", "Space Super-res. (\\%)"]]
mod_strats = {mod: pd.DataFrame([df.loc[mod] for df in strategy_dfs.values()]).reset_index(drop=True) for mod in
              strategy_dfs['Recurrent'].index}

def format_stats(stats):
    min_stat = sorted([stat['mean'] for stat in stats])[0]
    second_stat = sorted([stat['mean'] for stat in stats])[1]
    prec = 2
    for j in range(len(stats)):
        stat = stats[j]
        if stat['mean'] == min_stat:
            stat = f"$\\mathbf{{{round_(stat['mean'], prec)}}}({round_(stat['sd'], prec)})$"
        elif stat['mean'] == second_stat:
            stat = f"$\\underline{{{round_(stat['mean'], prec)}}}({round_(stat['sd'], prec)})$"
        else:
            stat = f"${round_(stat['mean'], prec)}({round_(stat['sd'], prec)})$"
        stats[j] = stat

for mod in mod_strats.keys():
    mod_strats[mod].set_index("Strategy", inplace=True)
    format_stats(mod_strats[mod]['Valid (\\%)'])
    format_stats(mod_strats[mod]["Space Super-res. (\\%)"])
df = pd.concat(mod_strats)
df.rename({"Space Super-res. (\\%)":"SR Test (\\%)"}, axis=1, inplace=True)

In [31]:
df.index = pd.MultiIndex.from_tuples([("\multirow{3}{*}{" + model + "}", strategy) for model, strategy in df.index])

In [32]:
print(df.to_latex(escape=False, multicolumn_format="c", column_format="llcc"))

\begin{tabular}{llcc}
\toprule
                                 &                 &                 Valid (\%) &                SR Test (\%) \\
\midrule
\multirow{3}{*}{FNO} & Markov &      $\mathbf{7.19}(0.28)$ &               $61.10(2.58)$ \\
                                 & Recurrent &              $16.23(0.49)$ &      $\mathbf{42.14}(0.25)$ \\
                                 & Teacher Forcing &   $\underline{8.64}(0.19)$ &   $\underline{43.02}(0.18)$ \\
\multirow{3}{*}{FNO+$p4$} & Markov &  $\underline{10.90}(0.24)$ &               $60.46(2.92)$ \\
                                 & Recurrent &              $17.60(0.14)$ &      $\mathbf{41.22}(0.41)$ \\
                                 & Teacher Forcing &     $\mathbf{10.70}(0.49)$ &   $\underline{49.78}(8.40)$ \\
\multirow{3}{*}{FNO+$p4m$} & Markov &     $\mathbf{18.26}(0.75)$ &               $60.41(2.28)$ \\
                                 & Recurrent &  $\underline{20.81}(0.28)$ &      $\mathbf{40.63}(0.20)$ \\
             

/tmp/ipykernel_451806/1292847556.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(escape=False, multicolumn_format="c", column_format="llcc"))


# Grid

In [33]:
len(exps['ns_cartesian']), len(exps['ns_symmetric']), len(exps['ns_nogrid'])

(15, 9, 24)

In [34]:
cartesian = exps['ns_cartesian'] + [exp for exp in exps['ns'] if 'FNO2d' in exp and 'G' not in exp]
len(cartesian)

24

In [35]:
symmetric = exps['ns_symmetric'] + [exp for exp in exps['ns'] if '_FNO' not in exp and '3d' not in exp and '3D' not in exp]
len(symmetric)

24

In [36]:
nogrid = exps['ns_nogrid']
len(nogrid)

24

In [37]:
exps.keys()
tune_keys = ["Valid", "Test", "Rotation Test"]

# collect experiments by grid
grid_mods = mods2d
grid_exps = {"None": nogrid, "Symmetric": symmetric, "Cartesian": cartesian}
grid_dfs = {
    grid: results_frame(exps, grid_mods, tune_keys, format=0, par_millions=True, remove_dimension=True) for
    grid, exps in grid_exps.items()}

In [38]:
grid_dfs['None']

,Valid (\%),Test (\%),Test\textsubscript{$90^\circ$} (\%)
FNO,"{'mean': 9.05258846282959, 'sd': 0.18974581456...","{'mean': 8.536127408345541, 'sd': 0.3613272927...","{'mean': 130.14233016967773, 'sd': 2.059915737..."
FNO+$p4$,"{'mean': 10.822073380152384, 'sd': 0.207053272...","{'mean': 10.457118590672811, 'sd': 0.305529490...","{'mean': 10.463987072308859, 'sd': 0.461810792..."
FNO+$p4m$,"{'mean': 23.540175755818684, 'sd': 0.573807472...","{'mean': 23.069985230763752, 'sd': 1.292431773...","{'mean': 23.6457618077596, 'sd': 1.04297725776..."
$G$-FNO-$p4$,"{'mean': 4.446248014767964, 'sd': 0.2735953713...","{'mean': 4.469608346621196, 'sd': 0.2074924420...","{'mean': 4.46962986389796, 'sd': 0.20748950785..."
$G$-FNO-$p4m$,"{'mean': 7.153340816497803, 'sd': 0.2533013744...","{'mean': 6.746675729751587, 'sd': 0.1724088242...","{'mean': 6.746685067812602, 'sd': 0.1723886786..."
U-Net-$p4$,"{'mean': 18.000698804855347, 'sd': 0.595628361...","{'mean': 17.94672171274821, 'sd': 0.0656687463...","{'mean': 17.947948058446247, 'sd': 0.063016575..."
radialFNO-$p4$,"{'mean': 9.560741504033407, 'sd': 0.3102194159...","{'mean': 9.131907939910889, 'sd': 0.0927397596...","{'mean': 9.131918668746948, 'sd': 0.0927499944..."
radialFNO-$p4m$,"{'mean': 11.893613656361898, 'sd': 0.336177588...","{'mean': 11.007163882255554, 'sd': 0.038575444...","{'mean': 11.007185180981955, 'sd': 0.038499021..."


In [39]:
grid_dfs['Symmetric']

,Valid (\%),Test (\%),Test\textsubscript{$90^\circ$} (\%)
FNO,"{'mean': 9.014573812484741, 'sd': 0.2602454404...","{'mean': 8.948764642079672, 'sd': 0.1833960657...","{'mean': 129.08466720581055, 'sd': 3.951227049..."
FNO+$p4$,"{'mean': 10.625682274500528, 'sd': 0.066405769...","{'mean': 11.040176590283712, 'sd': 0.260041133...","{'mean': 10.466412981351217, 'sd': 0.361382297..."
FNO+$p4m$,"{'mean': 23.67054057121277, 'sd': 1.3136283617...","{'mean': 23.286169687906902, 'sd': 0.439226961...","{'mean': 23.0897114276886, 'sd': 0.28569644931..."
$G$-FNO-$p4$,"{'mean': 4.8642130096753435, 'sd': 0.318567073...","{'mean': 4.781348029772441, 'sd': 0.3853128675...","{'mean': 4.781357069810231, 'sd': 0.3852913003..."
$G$-FNO-$p4m$,"{'mean': 6.727189421653748, 'sd': 0.8240622220...","{'mean': 6.186302085717519, 'sd': 0.6132987009...","{'mean': 6.18630172808965, 'sd': 0.61330382523..."
U-Net-$p4$,"{'mean': 18.862011591593426, 'sd': 0.526796956...","{'mean': 18.404366493225098, 'sd': 0.435045484...","{'mean': 18.404366493225098, 'sd': 0.435042889..."
radialFNO-$p4$,"{'mean': 9.59197203318278, 'sd': 0.05006147528...","{'mean': 9.205094655354818, 'sd': 0.2628118306...","{'mean': 9.205117424329122, 'sd': 0.2628167962..."
radialFNO-$p4m$,"{'mean': 11.51237948735555, 'sd': 0.6846004338...","{'mean': 10.861871441205343, 'sd': 0.183633212...","{'mean': 10.861878116925558, 'sd': 0.183648826..."


In [40]:
grid_dfs['Cartesian']

,Valid (\%),Test (\%),Test\textsubscript{$90^\circ$} (\%)
FNO,"{'mean': 8.639811833699545, 'sd': 0.1916359586...","{'mean': 8.410162965456644, 'sd': 0.4081481196...","{'mean': 129.21417427062988, 'sd': 3.898874899..."
FNO+$p4$,"{'mean': 10.700923681259155, 'sd': 0.493220610...","{'mean': 10.444285909334818, 'sd': 0.470016048...","{'mean': 10.377301494280497, 'sd': 0.382363717..."
FNO+$p4m$,"{'mean': 22.432045936584473, 'sd': 1.590954218...","{'mean': 22.088563442230225, 'sd': 1.462111046...","{'mean': 22.607455571492512, 'sd': 1.539465679..."
$G$-FNO-$p4$,"{'mean': 4.605937361717224, 'sd': 0.2532977321...","{'mean': 4.390383005142212, 'sd': 0.2480714893...","{'mean': 4.388887306054433, 'sd': 0.2457127486..."
$G$-FNO-$p4m$,"{'mean': 6.862679958343506, 'sd': 0.1738919711...","{'mean': 6.665010253588359, 'sd': 0.3163513237...","{'mean': 6.667257010936737, 'sd': 0.3214247618..."
U-Net-$p4$,"{'mean': 18.839237133661907, 'sd': 1.147875126...","{'mean': 17.73415207862854, 'sd': 0.1339453835...","{'mean': 18.06775776545207, 'sd': 0.2675799222..."
radialFNO-$p4$,"{'mean': 9.785064379374186, 'sd': 0.3100138107...","{'mean': 9.578201611836752, 'sd': 0.0540160009...","{'mean': 24.749619046847027, 'sd': 20.20425553..."
radialFNO-$p4m$,"{'mean': 11.361075520515442, 'sd': 0.444620301...","{'mean': 11.200425664583841, 'sd': 0.118494934...","{'mean': 11.83778707186381, 'sd': 1.0193949365..."


In [42]:
for grid in grid_exps.keys():
    grid_dfs[grid]['Positional Encoding'] = grid
    grid_dfs[grid] = grid_dfs[grid][["Positional Encoding", "Valid (\\%)", "Test (\\%)", "Test\\textsubscript{$90^\\circ$} (\\%)"]]
mod_grid = {mod: pd.DataFrame([df.loc[mod] for df in grid_dfs.values()]).reset_index(drop=True) for mod in
              grid_dfs['None'].index}
for mod in mod_grid.keys():
    mod_grid[mod].set_index("Positional Encoding", inplace=True)
    format_stats(mod_grid[mod]['Valid (\\%)'])
    format_stats(mod_grid[mod]['Test (\\%)'])
    format_stats(mod_grid[mod]['Test\\textsubscript{$90^\\circ$} (\\%)'])
df = pd.concat(mod_grid)
df.index = pd.MultiIndex.from_tuples([("\multirow{3}{*}{" + model + "}", grid) for model, grid in df.index])
print(df.to_latex(escape=False, multicolumn_format="c", column_format="llcc"))

\begin{tabular}{llcc}
\toprule
                                 &           &                 Valid (\%) &                  Test (\%) & Test\textsubscript{$90^\circ$} (\%) \\
\midrule
\multirow{3}{*}{FNO} & None &               $9.05(0.19)$ &   $\underline{8.54}(0.36)$ &                      $130.14(2.06)$ \\
                                 & Symmetric &   $\underline{9.01}(0.26)$ &               $8.95(0.18)$ &             $\mathbf{129.08}(3.95)$ \\
                                 & Cartesian &      $\mathbf{8.64}(0.19)$ &      $\mathbf{8.41}(0.41)$ &          $\underline{129.21}(3.90)$ \\
\multirow{3}{*}{FNO+$p4$} & None &              $10.82(0.21)$ &  $\underline{10.46}(0.31)$ &           $\underline{10.46}(0.46)$ \\
                                 & Symmetric &     $\mathbf{10.63}(0.07)$ &              $11.04(0.26)$ &                       $10.47(0.36)$ \\
                                 & Cartesian &  $\underline{10.70}(0.49)$ &     $\mathbf{10.44}(0.47)$ &              $\mathb

/tmp/ipykernel_451806/1997091089.py:13: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(escape=False, multicolumn_format="c", column_format="llcc"))


# Steerable

In [48]:
exps.keys()

dict_keys(['ns_sym', 'swe', 'ns', 'arena', 'ns_markov', 'ns_steer', 'ns_recurrent', 'ns_hybrid', 'ns_nogrid', 'ns_symmetric', 'ns_cartesian'])

In [49]:
len(exps['ns_steer'])

6

In [50]:
steerable_exps = exps['ns_steer']
gfno_ns_exps = [exp for exp in exps['ns'] if 'GFNO2d' in exp]

In [51]:
len(steerable_exps), len(gfno_ns_exps)

(6, 6)

In [52]:
steerable_mods = ['GFNO2d_p4_steer', 'GFNO2d_p4m_steer', 'GFNO2d_p4', 'GFNO2d_p4m']
keys_steer = ['Parameters', 'Test', 'Rotation Test']

In [53]:
steer = results_frame(steerable_exps + gfno_ns_exps, steerable_mods, keys_steer, prec=2, format=False,summary_only=True, remove_dimension=True, par_millions=True)

In [54]:
steer.index = [mod.replace('-p4-', '-$p4$-').replace('-p4m-', '-$p4m$-') for mod in list(steer.index)]

In [55]:
print(steer.to_latex(escape=False, column_format="lcc"))

\begin{tabular}{lcc}
\toprule
{} & \# Par. (M) &                 Test (\%) & Test\textsubscript{$90^\circ$} (\%) \\
\midrule
$G$-FNO-$p4$-steer  &      $0.83$ &             $20.87(1.25)$ &                       $20.87(1.25)$ \\
$G$-FNO-$p4m$-steer &      $0.89$ &             $22.58(0.41)$ &                       $22.58(0.41)$ \\
$G$-FNO-$p4$        &      $0.85$ &     $\mathbf{4.78}(0.39)$ &               $\mathbf{4.78}(0.39)$ \\
$G$-FNO-$p4m$       &      $0.84$ &  $\underline{6.19}(0.61)$ &            $\underline{6.19}(0.61)$ \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3755212/2369649096.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(steer.to_latex(escape=False, column_format="lcc"))


# Exp counts

In [146]:
exp_ct = {data:{model:len([exp for exp in exps[data] if "/" + model + "/" in exp]) for model in mods} for data in datas.keys()}
pd.DataFrame(exp_ct)

,ns_sym,swe,ns,arena
FNO2d,3,3,3,3
FNO2d_aug,3,3,3,3
FNO2d_aug-rf,3,3,3,3
FNO3d,3,3,3,3
FNO3d_aug,3,3,3,3
FNO3d_aug-rf,3,3,3,0
GFNO2d_p4,3,3,3,3
GFNO2d_p4m,3,3,3,3
GFNO3d_p4,3,3,3,3
GFNO3d_p4m,3,3,3,1


In [60]:
exp_ct = {data:{model:len([seed for seed in ['seed1', 'seed2', 'seed3'] if seed in " ".join([exp for exp in exps[data] if "/" + model + "/" in exp])]) for model in mods} for data in datas.keys()}
pd.DataFrame(exp_ct)

,ns_sym,swe,ns
FNO2d,3,3,3
FNO2d_aug,3,3,3
FNO2d_aug-rf,3,3,3
FNO3d,3,3,3
FNO3d_aug,3,3,3
FNO3d_aug-rf,3,3,3
GFNO2d_p4,3,3,3
GFNO2d_p4m,3,3,3
GFNO3d_p4,3,3,3
GFNO3d_p4m,3,3,3
